In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

In [7]:
data = gpd.read_file('OSM/armenia-latest-free.shp/gis_osm_transport_free_1.shp')
for file in tqdm(['belarus', 'kazakhstan', 'central-fed-district',
             'crimean-fed-district', 'far-eastern-fed-district', 'north-caucasus-fed-district',
             'northwestern-fed-district', 'siberian-fed-district', 'south-fed-district',
             'ural-fed-district', 'volga-fed-district']):

    data1 = gpd.read_file('OSM/%s-latest-free.shp/gis_osm_transport_free_1.shp' %file)
    data = pd.concat([data, data1], ignore_index=True)
    
    
    
data = gpd.GeoDataFrame(data, geometry='geometry')

100%|██████████| 11/11 [00:04<00:00,  2.21it/s]


In [99]:
new_data.to_file('railway_stations.gpkg', driver='GPKG')

In [8]:
new_data = data[data['fclass']=='railway_station']

In [11]:
polygon_file = "isochrones.gpkg"
polygons = gpd.read_file(polygon_file)

In [12]:
izo30 = polygons[polygons['iso_time'] == 30]
izo15 = polygons[polygons['iso_time'] == 15]
izo10 = polygons[polygons['iso_time'] == 10]
izo5 = polygons[polygons['iso_time'] == 5]
izo3 = polygons[polygons['iso_time'] == 3]

In [113]:
train = pd.read_csv('train.csv')
train['type'] = 'train'
test = pd.read_csv('test.csv')
test['type'] = 'test'
shops = pd.concat([train, test], ignore_index=True)


In [114]:
for num, izo in zip([3, 5, 10, 15, 30], [izo3, izo5, izo10, izo15, izo30]):
    points_in_polygons = gpd.sjoin(new_data, izo, how="inner", op="within")
    points_count_per_polygon = points_in_polygons.groupby(by='id').size()
    points_count_per_polygon.name = 'count_railway_station_iso_' + str(num)
    shops = shops.merge(points_count_per_polygon, how='left', on='id')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use th

In [123]:
shops.count_railway_station_iso_3.fillna(0, inplace = True)
shops.count_railway_station_iso_5.fillna(0, inplace = True)
shops.count_railway_station_iso_10.fillna(0, inplace = True)
shops.count_railway_station_iso_15.fillna(0, inplace = True)
shops.count_railway_station_iso_30.fillna(0, inplace = True)

In [125]:
new_train = shops[shops['type'] == 'train']
new_test = shops[shops['type'] == 'test']

new_train.to_csv('train_metro_count_per_pvz.csv', index = False)
new_test.to_csv('test_metro_count_per_pvz.csv', index = False)